In [ ]:
!pip install category_encoders
!pip install torch

In [1]:
import pandas as pd
import numpy as np
from category_encoders.ordinal import OrdinalEncoder
from sklearn.model_selection import train_test_split
from preprocess_custom import preprocess
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
import wandb

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [27]:
class MLP(nn.Module):
  def __init__(self):
    super(MLP, self).__init__()
    self.fc1 = nn.Linear(21, 128)
    self.fc2 = nn.Linear(128, 256)
    self.fc3 = nn.Linear(256, 112)
    self.fc4 = nn.Linear(112, 56)
    self.fc5 = nn.Linear(56, 1)
    self.relu = nn.ReLU()
    self.sigmoid = nn.Sigmoid()
    self.dropout = nn.Dropout(0.25)

  def forward(self, x):
    x = self.fc1(x)
    x = self.relu(x)
    x = self.dropout(x)
    x = self.fc2(x)
    x = self.relu(x)
    x = self.dropout(x)
    x = self.fc3(x)
    x = self.relu(x)
    x = self.dropout(x)
    x = self.fc4(x)
    x = self.relu(x)
    x = self.dropout(x)
    x = self.fc5(x)
    x = self.sigmoid(x)
    return x

def calculate_accuracy(outputs, targets):
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == targets).sum().item()
    accuracy = correct / targets.size(0)
    return accuracy

In [4]:
wandb.login()
wandb.init(project="mlp")

AttributeError: module 'wandb' has no attribute 'login'

In [7]:
# 데이터 읽기
df_train = pd.read_csv("data/train.csv")  # 학습용 데이터
df_test = pd.read_csv("data/submission.csv")  # 테스트 데이터(제출파일의 데이터)
df_test_id = df_test['id']
df_test_no_id = df_test.drop(columns=['id'])
# df_all = pd.concat([df_train, df_test_no_id]).reset_index(drop=True)
# 데이터 전처리
df_train, df_test = preprocess(df_train, df_test_no_id)

In [8]:
df_train

,bant_submit,customer_country,business_unit,customer_idx,customer_type,enterprise_Enterprise,enterprise_SMB,historical_existing_cnt,customer_job,lead_desc_length,...,customer_position,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,lead_owner,is_converted
0,1.00,1,1,1,1,1,0,0.0,1,62,...,1,1,1,1,0,0.003079,0.026846,1,1,True
1,1.00,2,1,2,1,1,0,12.0,2,96,...,2,1,1,1,0,0.003079,0.026846,1,2,True
2,1.00,3,1,3,1,1,0,144.0,3,56,...,3,2,1,1,0,0.003079,0.026846,1,3,True
3,1.00,4,1,4,1,1,0,0.0,4,44,...,2,2,1,1,0,0.003079,0.026846,1,4,True
4,1.00,5,1,5,2,1,0,0.0,5,97,...,3,2,1,0,0,0.003079,0.026846,1,5,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59294,1.00,5108,1,35108,6,0,1,0.0,3,200,...,1,28,3,0,0,0.000026,0.028777,12,628,False
59295,0.75,2,1,35109,2,1,0,0.0,4,70,...,12,10,4,0,0,0.000026,0.028777,12,40,False
59296,0.75,2,1,35110,2,1,0,0.0,3,34,...,1,24,1,0,0,0.000026,0.028777,12,122,False
59297,1.00,2,1,35111,30,1,0,0.0,4,377,...,13,24,5,0,0,0.000026,0.028777,12,126,False


In [9]:
# "is_converted" 열에 대해 True를 1로, False를 0으로 변환
df_train["is_converted"] = df_train["is_converted"].replace({True: 1, False: 0})
df_test["is_converted"] = df_test["is_converted"].replace({True: 1, False: 0})
df_train

,bant_submit,customer_country,business_unit,customer_idx,customer_type,enterprise_Enterprise,enterprise_SMB,historical_existing_cnt,customer_job,lead_desc_length,...,customer_position,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,lead_owner,is_converted
0,1.00,1,1,1,1,1,0,0.0,1,62,...,1,1,1,1,0,0.003079,0.026846,1,1,1
1,1.00,2,1,2,1,1,0,12.0,2,96,...,2,1,1,1,0,0.003079,0.026846,1,2,1
2,1.00,3,1,3,1,1,0,144.0,3,56,...,3,2,1,1,0,0.003079,0.026846,1,3,1
3,1.00,4,1,4,1,1,0,0.0,4,44,...,2,2,1,1,0,0.003079,0.026846,1,4,1
4,1.00,5,1,5,2,1,0,0.0,5,97,...,3,2,1,0,0,0.003079,0.026846,1,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59294,1.00,5108,1,35108,6,0,1,0.0,3,200,...,1,28,3,0,0,0.000026,0.028777,12,628,0
59295,0.75,2,1,35109,2,1,0,0.0,4,70,...,12,10,4,0,0,0.000026,0.028777,12,40,0
59296,0.75,2,1,35110,2,1,0,0.0,3,34,...,1,24,1,0,0,0.000026,0.028777,12,122,0
59297,1.00,2,1,35111,30,1,0,0.0,4,377,...,13,24,5,0,0,0.000026,0.028777,12,126,0


In [11]:
# 수치형 데이터 스케일링
numeric_col = ["bant_submit", "customer_idx", "historical_existing_cnt", "lead_desc_length", 
               "ver_cus", "ver_pro", "ver_win_rate_x", "ver_win_ratio_per_bu", 
            "lead_owner", "is_converted"]
scaler = MinMaxScaler()
df_train[numeric_col] = scaler.fit_transform(df_train[numeric_col])
df_test[numeric_col] = scaler.transform(df_test[numeric_col])


In [12]:
df_train

,bant_submit,customer_country,business_unit,customer_idx,customer_type,enterprise_Enterprise,enterprise_SMB,historical_existing_cnt,customer_job,lead_desc_length,...,customer_position,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,lead_owner,is_converted
0,1.00,1,1,0.000000,1,1,0,0.000000,1,0.048298,...,1,1,1,1.0,0.0,1.000000,0.093960,1,0.000000,1.0
1,1.00,2,1,0.000028,1,1,0,0.008608,2,0.075218,...,2,1,1,1.0,0.0,1.000000,0.093960,1,0.001017,1.0
2,1.00,3,1,0.000057,1,1,0,0.103300,3,0.043547,...,3,2,1,1.0,0.0,1.000000,0.093960,1,0.002035,1.0
3,1.00,4,1,0.000085,1,1,0,0.000000,4,0.034046,...,2,2,1,1.0,0.0,1.000000,0.093960,1,0.003052,1.0
4,1.00,5,1,0.000114,2,1,0,0.000000,5,0.076010,...,3,2,1,0.0,0.0,1.000000,0.093960,1,0.004069,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59294,1.00,5108,1,0.999886,6,0,1,0.000000,3,0.157561,...,1,28,3,0.0,0.0,0.008408,0.100719,12,0.637843,0.0
59295,0.75,2,1,0.999915,2,1,0,0.000000,4,0.054632,...,12,10,4,0.0,0.0,0.008408,0.100719,12,0.039674,0.0
59296,0.75,2,1,0.999943,2,1,0,0.000000,3,0.026128,...,1,24,1,0.0,0.0,0.008408,0.100719,12,0.123093,0.0
59297,1.00,2,1,0.999972,30,1,0,0.000000,4,0.297704,...,13,24,5,0.0,0.0,0.008408,0.100719,12,0.127162,0.0


In [13]:

x_train, x_val, y_train, y_val = train_test_split(
    df_train.drop("is_converted", axis=1),
    df_train["is_converted"],
    test_size=0.1,
    shuffle=True,
    random_state=400,
)

x_train_tensor = torch.tensor(x_train.values, dtype=torch.float32)
x_val_tensor = torch.tensor(x_val.values, dtype=torch.float32)
df_test = df_test.drop("is_converted", axis=1)
x_test_tensor = torch.tensor(df_test.values, dtype=torch.float32)

y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32)

dataset = TensorDataset(x_train_tensor, y_train_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [30]:
model = MLP()

criterion = nn.BCELoss()

optimizer = optim.Adam(model.parameters(), lr=0.0001)
# 학습
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [34]:
num_epochs = 100
for epoch in range(1, num_epochs+1):
    total_loss = 0.0
    total_accuracy = 0.0

    for batch_x, batch_y in dataloader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)

        outputs = model(batch_x)
        batch_y = batch_y.unsqueeze(1)
        loss = criterion(outputs, batch_y)
        total_loss += loss.item()

        accuracy = calculate_accuracy(outputs, batch_y)
        total_accuracy += accuracy

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(dataloader)
    avg_accuracy = total_accuracy / len(dataloader)

    if epoch % 10 == 0:
        print('Epoch [{}/{}], Loss: {}, Accuracy: {}'.format(
        epoch, num_epochs, avg_loss, avg_accuracy
    ))


Epoch [10/100], Loss: 0.2696100905564406, Accuracy: 29.36810551558753
Epoch [20/100], Loss: 0.24157922612882846, Accuracy: 29.36810551558753
Epoch [30/100], Loss: 0.21734228352089222, Accuracy: 29.36810551558753
Epoch [40/100], Loss: 0.19450599699542462, Accuracy: 29.36810551558753
Epoch [50/100], Loss: 0.18378456002011967, Accuracy: 29.36810551558753
Epoch [60/100], Loss: 0.17649008105207475, Accuracy: 29.36810551558753
Epoch [70/100], Loss: 0.17082305195042305, Accuracy: 29.36810551558753
Epoch [80/100], Loss: 0.16768343474188868, Accuracy: 29.36810551558753
Epoch [90/100], Loss: 0.16471050454669897, Accuracy: 29.36810551558753
Epoch [100/100], Loss: 0.1616657660342753, Accuracy: 29.36810551558753


In [68]:
# eval
model.eval()

with torch.no_grad():
    x_test_tensor = x_test_tensor.to(device)
    print(x_test_tensor.shape)
    predictions = model(x_test_tensor)
    
predicted_labels = (predictions > 0.5).squeeze().tolist()
print(len(predicted_labels))
# 0과 1의 개수를 세기
count_zeros = predicted_labels.count(True)
count_ones = predicted_labels.count(False)
print("0의 개수:", count_zeros)
print("1의 개수:", count_ones)

torch.Size([5271, 21])
5271
0의 개수: 624
1의 개수: 4647


In [ ]:
predicted_labels

In [70]:
# NumPy 배열을 DataFrame으로 변환
df_predicted = pd.DataFrame(predicted_labels, columns=["is_converted"])

In [73]:
df_predicted

,is_converted
0,False
1,False
2,False
3,False
4,False
...,...
5266,False
5267,False
5268,False
5269,False


In [71]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("data/submission.csv")
df_sub["is_converted"] = df_predicted
# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

In [72]:
df_sub

,id,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,19844,0.00,/ / Brazil,ID,0.073248,47466,End Customer,Enterprise,53.0,NaN,...,LGESP,NaN,1,0,0.001183,0.049840,retail,Electronics & Telco,278,False
1,9738,0.25,400 N State Of Franklin Rd Cloud IT / Johnson...,IT,NaN,5405,End Customer,SMB,NaN,NaN,...,LGEUS,NaN,0,0,0.000013,NaN,transportation,Others,437,False
2,8491,1.00,/ / U.A.E,ID,NaN,13597,Specifier/ Influencer,SMB,NaN,NaN,...,LGEGF,less than 3 months,0,0,0.000060,0.131148,hospital & health care,General Hospital,874,False
3,19895,0.50,/ Madison / United States,ID,0.118644,17204,NaN,Enterprise,NaN,NaN,...,LGEUS,more than a year,0,0,0.001183,0.049840,retail,NaN,194,False
4,10465,1.00,/ Sao Paulo / Brazil,ID,0.074949,2329,End Customer,Enterprise,2.0,1.0,...,LGESP,less than 3 months,1,1,0.003079,0.064566,corporate / office,Engineering,167,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5266,13855,0.50,/São Paulo/Brazil,AS,NaN,40292,NaN,Enterprise,10.0,NaN,...,LGESP,NaN,0,0,NaN,NaN,NaN,NaN,97,False
5267,7979,0.25,General / / United States,IT,NaN,47466,NaN,Enterprise,0.0,NaN,...,LGEUS,NaN,0,0,NaN,NaN,NaN,NaN,438,False
5268,12887,0.75,/ OURO BRANCO / Brazil,AS,NaN,46227,Specifier/ Influencer,Enterprise,NaN,NaN,...,LGESP,less than 3 months,0,0,NaN,NaN,NaN,NaN,97,False
5269,17530,0.00,/ / Germany,IT,NaN,45667,End Customer,SMB,NaN,NaN,...,LGEDG,NaN,0,0,NaN,NaN,NaN,NaN,429,False
